# Word2Vec con Gensim

## Instalación de librerías y carga de dataset

In [1]:
import warnings
warnings.filterwarnings('ignore')
from gensim.models import Word2Vec
import pandas as pd
import re
from gensim.parsing.preprocessing import strip_punctuation, strip_numeric, strip_short, stem_text
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize

-------------------------

In [2]:
from datasets import load_dataset

dataset_corpus = load_dataset("large_spanish_corpus", "ParaCrawl")

Generating train split: 100%|██████████| 15510649/15510649 [01:41<00:00, 153033.32 examples/s]


In [ ]:
dataset_corpus

In [ ]:
subset = dataset_corpus['train'].select(range(1000000))

In [ ]:
subset[0:2]

In [ ]:
import nltk
nltk.download('stopwords')
nltk.download('punkt')

from nltk.tokenize import word_tokenize

## Pre-procesamiento de texto

In [ ]:
def clean_text(sentence_batch):
    # extrae el texto de la entrada
    text_list = sentence_batch['text']

    cleaned_text_list = []
    for text in text_list:
        # Convierte el texto a minúsculas
        text = text.lower()

        # Elimina URLs
        text = re.sub(r'http\S+|www\S+|https\S+', '', text, flags=re.MULTILINE)

        # Elimina las menciones @ y '#' de las redes sociales
        text = re.sub(r'\@\w+|\#\w+', '', text)

        # Elimina los caracteres de puntuación
        text = strip_punctuation(text)

        # Elimina los números
        text = strip_numeric(text)

        # Elimina las palabras cortas
        text = strip_short(text,minsize=2)

        # Elimina las palabras comunes (stop words)
        stop_words = set(stopwords.words('spanish'))
        word_tokens = word_tokenize(text)
        filtered_text = [word for word in word_tokens if word not in stop_words]

        cleaned_text_list.append(filtered_text)

    # Devuelve el texto limpio
    return {'text': cleaned_text_list}

In [ ]:
sentences_corpus = subset.map(clean_text, batched=True)

In [ ]:
sentences_corpus['text'][:3]

## Carga y uso de modelo de embeddings Word2Vec

In [ ]:
model = Word2Vec(sentences_corpus['text'], vector_size=100, window=5, min_count= 2, workers=6, sg=1)

# Podemos guardar el modelo para uso futuro
model.save("word2vec.model")

In [ ]:
model.wv['rey']

In [ ]:
##comida, ser, reina, television
model.wv.most_similar(['television'],topn=3)

In [ ]:
word_vectors = model.wv
vectors = word_vectors.vectors
words = word_vectors.index_to_key

## Almacenamiento de embeddings

In [ ]:
df_vectors = pd.DataFrame(vectors)
df_vectors.to_csv('embeddings.tsv',sep='\t', index=False)

In [ ]:
df_words = pd.DataFrame(words)
df_words.to_csv('labels.tsv',sep='\t', index=False)